# **Homework 2**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import requests, urllib
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [ ]:
url = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv"
resp = requests.get(url)
car_fuel_efficiency_data = resp.content

In [ ]:
csv_file = open('car_fuel_efficiency.csv', 'wb')
csv_file.write(car_fuel_efficiency_data)